In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split


import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
propiedades = pd.read_csv('../../set_datos_propiedades.csv')

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

# Decision Tree

In [14]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [15]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [16]:
tree = DecisionTreeRegressor()
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 90.99 % , error = 12434347138.596506


## Ahora que tenemos una intuicion, probamos cambiando los parametros

In [7]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [8]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

res = []

In [ ]:
lista_criterio = ['mse','friedman_mse']
lista_max_depth = [None,1,5,10]
lista_min_samples_split = [2,3]
lista_min_samples_leaf = [1,10]
lista_features = [0.4,0.6,1.0]

for criterio in lista_criterio:
    for max_depth in lista_max_depth:
        for min_sample in lista_min_samples_split:
            for min_leaf in lista_min_samples_leaf:
                for feature in lista_features:
                    
                    tree = DecisionTreeRegressor(criterion = criterio, max_depth = max_depth, min_samples_split = min_sample,\
                                    min_samples_leaf = min_leaf, max_features = feature)
                    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
                    set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply\
                            (lambda x: tree.predict(x)[0],axis = 1)
                    precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
                    error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
                    res.append((criterio,max_depth,min_sample,min_leaf,feature,precision,error))
                    print(criterio,' - ',max_depth,' - ',min_sample,' - ',min_leaf,' - ',feature)

In [10]:
for r in res:
    print("Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}"\
          .format(r[0],r[1],r[2],r[3],r[4],r[5],r[6]))

Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.4, precision = 90.92 % , error = 12523178385.446316
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.6, precision = 90.84 % , error = 12633432697.540848
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 1.0, precision = 90.99 % , error = 12436424287.069954
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 0.4, precision = 61.57 % , error = 53024760632.939766
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 0.6, precision = 58.74 % , error = 56920716118.86883
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 1.0, precision = 62.36 % , error = 51921658182.3563
Criterio = mse, max_depth = None, min_sample_split = 3, min_sample_leaf = 1, features = 0.4, precision = 90.29 % , error = 13398537273.276394
Criter

In [11]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[6] < min_error:
        min_error = r[6]
        tupla_min_error = r
    if r[5] > max_precision:
        max_precision = r[5]
        tupla_max_precision = r
        
print("Mayor precision = Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3],tupla_max_precision[4],tupla_max_precision[5],tupla_max_precision[6]))
print("Menor error = Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3],tupla_min_error[4],tupla_min_error[5],tupla_min_error[6]))

Mayor precision = Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 1.0, precision = 90.99 % , error = 12436424287.069954
Menor error = Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 1.0, precision = 90.99 % , error = 12436424287.069954


## Tomamos criterio = mse, max_depth = None ,min_sample = 2 ,min_leaf = 1, features = 1.0

In [56]:
#Probamos distintas combinaciones de columnas, esta es la que dio mejores resultados

columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio','aire','pileta','cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [57]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [58]:
criterio = 'mse'
max_depth = None
min_sample = 2
min_leaf = 1
feature = 1.0

In [59]:
tree = DecisionTreeRegressor(criterion = criterio, max_depth = max_depth, min_samples_split = min_sample,\
                                    min_samples_leaf = min_leaf, max_features = feature)
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 90.97 % , error = 12458466712.821684


# calculo con los verdaderos datos a analizar

In [60]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")

In [62]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)

In [63]:
resultado = analizar.loc[:,['id','price_usd']]

In [64]:
resultado.to_csv('Decision_tree_resultados', index = False)

# Ahora uso lat y lon en lugar de place name

In [2]:
propiedades = pd.read_csv('../../set_datos_propiedades.csv')

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()),\
                             ['lat', 'lon', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [5]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [6]:
tree = DecisionTreeRegressor()
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 98.28 % , error = 1989833789.7358894


In [7]:
res = []

In [ ]:
lista_criterio = ['mse','friedman_mse']
lista_max_depth = [None,1,5,10]
lista_min_samples_split = [2,3]
lista_min_samples_leaf = [1,10]
lista_features = [0.4,0.6,1.0]

for criterio in lista_criterio:
    for max_depth in lista_max_depth:
        for min_sample in lista_min_samples_split:
            for min_leaf in lista_min_samples_leaf:
                for feature in lista_features:
                    
                    tree = DecisionTreeRegressor(criterion = criterio, max_depth = max_depth, min_samples_split = min_sample,\
                                    min_samples_leaf = min_leaf, max_features = feature)
                    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
                    set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply\
                            (lambda x: tree.predict(x)[0],axis = 1)
                    precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
                    error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
                    res.append((criterio,max_depth,min_sample,min_leaf,feature,precision,error))
                    print(criterio,' - ',max_depth,' - ',min_sample,' - ',min_leaf,' - ',feature)

In [9]:
for r in res:
    print("Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}"\
          .format(r[0],r[1],r[2],r[3],r[4],r[5],r[6]))

Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.4, precision = 98.05 % , error = 2255960804.956864
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.6, precision = 98.28 % , error = 1989025537.2791593
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 1.0, precision = 98.21 % , error = 2074825073.4398386
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 0.4, precision = 64.17 % , error = 41481380227.237434
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 0.6, precision = 57.61 % , error = 49080302558.42927
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 1.0, precision = 63.76 % , error = 41963364067.33917
Criterio = mse, max_depth = None, min_sample_split = 3, min_sample_leaf = 1, features = 0.4, precision = 95.81 % , error = 4850759665.253196
Criteri

In [10]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[6] < min_error:
        min_error = r[6]
        tupla_min_error = r
    if r[5] > max_precision:
        max_precision = r[5]
        tupla_max_precision = r
        
print("Mayor precision = Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3],tupla_max_precision[4],tupla_max_precision[5],tupla_max_precision[6]))
print("Menor error = Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3],tupla_min_error[4],tupla_min_error[5],tupla_min_error[6]))

Mayor precision = Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.6, precision = 98.28 % , error = 1989025537.2791593
Menor error = Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.6, precision = 98.28 % , error = 1989025537.2791593


## Tomamos criterio = mse, max_depth = None ,min_sample = 2 ,min_leaf = 1, features = 0.6

In [49]:
#Probamos distintas combinaciones de columnas, esta es la que dio mejores resultados

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [50]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [51]:
criterio = 'mse'
max_depth = None
min_sample = 2
min_leaf = 1
feature = 0.6

In [52]:
tree = DecisionTreeRegressor(criterion = criterio, max_depth = max_depth, min_samples_split = min_sample,\
                                    min_samples_leaf = min_leaf, max_features = feature)
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 98.10 % , error = 2198804392.142215


# calculo con los verdaderos datos a analizar

In [53]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")

In [54]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)

In [55]:
resultado = analizar.loc[:,['id','price_usd']]

In [56]:
resultado.to_csv('resultados/Decision_tree_2', index = False)

# Usando Grid Search y Cross Validation

In [2]:
propiedades = pd.read_csv('../../set_datos_propiedades.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [3]:
# Parametros

criterios = ['mse','friedman_mse']
max_depth = [None,1,5,10]
samples_split = [2,3]
features = [0.4,0.6,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, \
              "min_samples_split" : samples_split, "max_features" : features}

iteraciones_cross_validation = 75
arbol = DecisionTreeRegressor()

In [4]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = -1, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 17:47:24 --- 19:17:30 
 Precision: 95.69 
 Parametros = {'criterion': 'mse', 'max_depth': None, 'max_features': 0.4, 'min_samples_split': 3}


In [5]:
#Con 10 iteraciones de CV: Precision: 92.52       12:17:11 --- 12:29:12
#               Parametros = {'criterion': 'mse', 'max_depth': None, 'max_features': 0.6, 'min_samples_split': 3}

#Con 30 iteraciones de CV:  Precision: 94.97       11:24:03 --- 12:00:09
#              Parametros = {'criterion': 'mse', 'max_depth': None, 'max_features': 0.4, 'min_samples_split': 2}

#Con 50 iteraciones de CV:    Precision: 95.61     12:09:55 --- 13:12:40 
#              Parametros = {'criterion': 'mse', 'max_depth': None, 'max_features': 0.6, 'min_samples_split': 2}

In [6]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_GridSearch.csv', index = False)

In [7]:
joblib.dump(grid, 'algoritmos/decision_tree.pkl')

['algoritmos/decision_tree.pkl']

## Hago bagging con el mejor algoritmo

In [2]:
propiedades = pd.read_csv('../../set_datos_propiedades.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_pruebas = analizar = pd.read_csv("../properati_dataset_modificado.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [3]:
cant = 10

for i in range(cant):
    datos = propiedades.sample(frac = 0.4)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = None, max_features = 0.4, min_samples_split = 3)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']

In [4]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_Cross_Validation.csv', index = False)

# Usando Grid Search y Cross Validation con las fechas

In [3]:
propiedades = pd.read_csv('../../set_datos_propiedades.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

propiedades.loc[:,'fecha'] = propiedades.loc[:,['Year', 'Month']].apply(lambda x: int(str(x[0]) + str(x[1])), axis = 1)

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire','pileta', 'cochera', 'fecha']
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [14]:
# Parametros

criterios = ['mse','friedman_mse']
max_depth = [None,5,10]
features = [0.4,0.6,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 25
arbol = DecisionTreeRegressor()

In [15]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 20:55:33 --- 21:14:42 
 Precision: 94.61 
 Parametros = {'criterion': 'friedman_mse', 'max_depth': None, 'max_features': 0.6}


In [17]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")

analizar.loc[:,'Month'] = analizar.created_on.apply(lambda x: int(x.split('-')[1]))
analizar.loc[:,'Year'] = analizar.created_on.apply(lambda x: int(x.split('-')[0]))
analizar.loc[:,'fecha'] = analizar.loc[:,['Year', 'Month']].apply(lambda x: int(str(x[0]) + str(x[1])), axis = 1)

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_GridSearch_fechas.csv', index = False)

In [18]:
joblib.dump(grid, 'algoritmos/decision_tree_3.pkl')

['algoritmos/decision_tree_3.pkl']

## Hago bagging con el mejor algoritmo

In [19]:
analizar.loc[:,'price_usd'] = 0.0

In [20]:
cant = 10

for i in range(cant):
    datos = propiedades.sample(frac = 0.4)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'friedman_mse', max_depth = None, max_features = 0.6)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    analizar.loc[:,'res'] = analizar.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
    analizar.loc[:,'price_usd'] = analizar.loc[:,'price_usd'] + analizar.loc[:,'res']

In [21]:
analizar.loc[:,'price_usd'] = analizar.loc[:,'price_usd'] / cant

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_Cross_Validation_fechas.csv', index = False)

# Usando Grid Search y Cross Validation con mas columnas

In [6]:
propiedades = pd.read_csv('../../set_datos_propiedades.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', \
            'pileta', 'cochera','transporte','comercio', 'servicios', 'doble piso', 'jardin']
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [7]:
# Parametros

criterios = ['mse','friedman_mse']
max_depth = [None,60]
features = [0.4,0.6,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 10
arbol = DecisionTreeRegressor()

In [8]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 18:43:39 --- 18:48:36 
 Precision: 92.76 
 Parametros = {'max_features': 1.0, 'criterion': 'mse', 'max_depth': 60}


In [10]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict([x])[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_GridSearch_mas_columnas.csv', index = False)

## Hago bagging con el mejor algoritmo

In [12]:
propiedades = pd.read_csv('../..set_datos_propiedades.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', \
            'pileta', 'cochera','transporte','comercio', 'servicios', 'doble piso', 'jardin']
columnas_precio = columnas + ['price_aprox_usd']

set_pruebas = analizar = pd.read_csv("../properati_dataset_modificado.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [15]:
cant = 10

for i in range(cant):
    datos = propiedades.sample(frac = 0.7)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 60, max_features = 1.0)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']

In [16]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_Cross_Validation_mas_columnas.csv', index = False)

# Usando SVD con varias dimensiones

In [2]:
propiedades = pd.read_csv('../../set_datos_propiedades.csv')

sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [3]:
X = propiedades.loc[:,columnas]
y = propiedades.loc[:,'price_aprox_usd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

for i in range(2,len(columnas)):
    svd = TruncatedSVD(n_components = i)
    datos = svd.fit_transform(X_train)
    test = svd.transform(X_test)
    
    tree = DecisionTreeRegressor()
    tree.fit(datos,y_train)
    precision = tree.score(test,y_test) * 100
    error = mean_squared_error(tree.predict(test),y_test)
    
    print("Dim = {}, Precision = {:.2f}, error = {}, digitos = {}".\
                  format(i,precision,error,len(str(int(round(error,0))))))

Dim = 2, Precision = 94.14, error = 730878798.442, digitos = 9
Dim = 3, Precision = 95.24, error = 593020063.06, digitos = 9
Dim = 4, Precision = 95.39, error = 575166124.756, digitos = 9
Dim = 5, Precision = 95.42, error = 571276210.332, digitos = 9
Dim = 6, Precision = 95.64, error = 543869222.969, digitos = 9
Dim = 7, Precision = 95.51, error = 560562059.834, digitos = 9
Dim = 8, Precision = 95.54, error = 556312307.442, digitos = 9


In [6]:
set_pruebas = analizar = pd.read_csv("../properati_dataset_modificado.csv")

svd = TruncatedSVD(n_components = 6)
datos = svd.fit_transform(propiedades.loc[:,columnas])
test = svd.transform(set_pruebas.loc[:,columnas])

tree = DecisionTreeRegressor()
tree.fit(datos,propiedades.loc[:,'price_aprox_usd'])
set_pruebas.loc[:,'price_usd'] = tree.predict(test)

resultado = set_pruebas.loc[:,['id','price_usd']]
resultado.to_csv('resultados/Decision_Tree_SVD.csv', index = False)

# Agregando el resultado de SVD dimension 2 al Dataframe

In [7]:
propiedades = pd.read_csv('../../set_datos_propiedades.csv')

sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

prop = propiedades.loc[(propiedades.Year >= 2016), :]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [8]:
set_entrenamiento = prop.loc[(prop.Year >= 2016) &((prop.Year < 2017) | (prop.Month < 6))\
                                    ,columnas_precio]
set_pruebas = prop.loc[(prop.Year == 2017) & (prop.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [9]:
svd = TruncatedSVD()
resultado = svd.fit_transform(set_entrenamiento_datos)
resultado = pd.DataFrame(resultado)
pruebas = svd.transform(set_pruebas.loc[:,columnas])
pruebas = pd.DataFrame(pruebas)

In [10]:
set_entrenamiento_datos.reset_index(drop = True, inplace = True)
set_entrenamiento_datos.loc[:,'SVD 1'] = resultado.loc[:,0]
set_entrenamiento_datos.loc[:,'SVD 2'] = resultado.loc[:,1]
set_entrenamiento_datos['SVD 1'].describe()

count    334935.000000
mean         95.098990
std          24.394078
min          52.920358
25%          75.951302
50%          88.599928
75%         110.070518
max         157.152840
Name: SVD 1, dtype: float64

In [11]:
set_pruebas.reset_index(drop = True, inplace = True)
set_pruebas.loc[:,'SVD 1'] = pruebas.loc[:,0]
set_pruebas.loc[:,'SVD 2'] = pruebas.loc[:,1]
set_pruebas['SVD 1'].describe()

count    20000.000000
mean        96.581427
std         24.831401
min         52.920358
25%         76.701836
50%         90.073607
75%        112.379512
max        157.152840
Name: SVD 1, dtype: float64

In [12]:
columnas = columnas + ['SVD 1', 'SVD 2']

In [19]:
criterio = 'mse'
max_depth = None
min_sample = 2
min_leaf = 1
feature = 0.6

tree = DecisionTreeRegressor(criterion = criterio, max_depth = max_depth, min_samples_split = min_sample,\
                                    min_samples_leaf = min_leaf, max_features = feature)
tree.fit(set_entrenamiento_datos, set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 96.92 % , error = 428205209.64


In [21]:
res = []

In [ ]:
lista_criterio = ['mse','friedman_mse']
lista_max_depth = [None,1,5,10]
lista_min_samples_split = [2,3]
lista_min_samples_leaf = [1,10]
lista_features = [0.4,0.6,1.0]

for criterio in lista_criterio:
    for max_depth in lista_max_depth:
        for min_sample in lista_min_samples_split:
            for min_leaf in lista_min_samples_leaf:
                for feature in lista_features:
                    
                    tree = DecisionTreeRegressor(criterion = criterio, max_depth = max_depth, min_samples_split = min_sample,\
                                    min_samples_leaf = min_leaf, max_features = feature)
                    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
                    set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply\
                            (lambda x: tree.predict([x])[0],axis = 1)
                    precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
                    error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
                    res.append((criterio,max_depth,min_sample,min_leaf,feature,precision,error))
                    print(criterio,' - ',max_depth,' - ',min_sample,' - ',min_leaf,' - ',feature)

In [23]:
for r in res:
    print("Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}"\
          .format(r[0],r[1],r[2],r[3],r[4],r[5],r[6]))

Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.4, precision = 96.78 % , error = 448563759.129
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.6, precision = 96.64 % , error = 467097985.363
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 1.0, precision = 96.85 % , error = 437849809.241
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 0.4, precision = 89.00 % , error = 1530772522.34
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 0.6, precision = 89.50 % , error = 1461564141.62
Criterio = mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 10, features = 1.0, precision = 90.44 % , error = 1331337408.24
Criterio = mse, max_depth = None, min_sample_split = 3, min_sample_leaf = 1, features = 0.4, precision = 96.87 % , error = 435233903.01
Criterio = mse, max_depth = None, min_s

In [24]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[6] < min_error:
        min_error = r[6]
        tupla_min_error = r
    if r[5] > max_precision:
        max_precision = r[5]
        tupla_max_precision = r
        
print("Mayor precision = Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3],tupla_max_precision[4],tupla_max_precision[5],tupla_max_precision[6]))
print("Menor error = Criterio = {}, max_depth = {}, min_sample_split = {}, min_sample_leaf = {}, features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3],tupla_min_error[4],tupla_min_error[5],tupla_min_error[6]))

Mayor precision = Criterio = friedman_mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.4, precision = 97.16 % , error = 395590550.85
Menor error = Criterio = friedman_mse, max_depth = None, min_sample_split = 2, min_sample_leaf = 1, features = 0.4, precision = 97.16 % , error = 395590550.85


In [25]:
criterio = 'friedman_mse'
max_depth = None
min_sample = 2
min_leaf = 1
feature = 0.4

In [33]:
tree = DecisionTreeRegressor(criterion = criterio, max_depth = max_depth, min_samples_split = min_sample,\
                                    min_samples_leaf = min_leaf, max_features = feature)
tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
precision = tree.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 97.17 % , error = 394167047.417


# calculo con los verdaderos datos a analizar

In [34]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")

In [35]:
columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
analisis = svd.transform(analizar.loc[:,columnas])
analisis = pd.DataFrame(analisis)

In [36]:
analizar.loc[:,'SVD 1'] = analisis.loc[:,0]
analizar.loc[:,'SVD 2'] = analisis.loc[:,1]
analizar['SVD 1'].describe()

count    1.416600e+04
mean     3.301517e+02
std      2.023766e+04
min      4.621164e+01
25%      8.041788e+01
50%      1.004089e+02
75%      1.568249e+02
max      2.405964e+06
Name: SVD 1, dtype: float64

In [37]:
columnas += ['SVD 1', 'SVD 2']
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)

In [38]:
analizar.price_usd.describe()

count      14166.000000
mean      193623.159905
std       147566.773996
min         5000.000000
25%        97500.000000
50%       150000.000000
75%       237950.000000
max      1000000.000000
Name: price_usd, dtype: float64

In [39]:
resultado = analizar.loc[:,['id','price_usd']]

In [40]:
resultado.to_csv('resultados/Decision_tree_menos_datos_svd_agregada', index = False)

# Grid Search y Cross Validation definitivo

In [2]:
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

In [3]:
propiedades = pd.read_csv('../../datos_con_fecha.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', \
            'pileta', 'cochera','transporte','comercio', 'servicios', 'doble piso', 'jardin','fecha']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [4]:
# Parametros

criterios = ['mse','friedman_mse']
max_depth = [None,50,75,100,150]
features = [0.4,0.6,0.8,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 12
arbol = DecisionTreeRegressor()

In [5]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 22:41:48 --- 22:58:00 
 Precision: 95.80 
 Parametros = {'criterion': 'mse', 'max_depth': 100, 'max_features': 0.8}


In [6]:
analizar = pd.read_csv("../properati_dataset_modificado2.csv")

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_GridSearch_definitivo.csv', index = False)
joblib.dump(grid, 'algoritmos/decision_tree_definitivo.pkl')

['algoritmos/decision_tree_definitivo.pkl']

# Haciendo bagging

In [9]:
propiedades = pd.read_csv('../../datos_con_fecha.csv')

sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

set_pruebas = pd.read_csv("../properati_dataset_modificado2.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [10]:
cant = 10
col = ['superficie','lat', 'lon','property_type_encoded','fecha']
comodidades = ['seguridad','gimnasio','aire','pileta','cochera','transporte','comercio','servicios','doble piso','jardin']

for i in range(cant):
    com = list(np.random.choice(comodidades,6,replace = False))
    columnas = col + com
    set_entrenamiento_datos = propiedades.loc[:,columnas]
    set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict(x)[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']
    print(i,end = ' - ')

0 - 1 - 2 - 3 - 4 - 5 - 6 - 7 - 8 - 9 - 

In [11]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_bagging_definitivo.csv', index = False)

# Grid Search y Cross Validation definitivo sin fecha

In [2]:
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

In [3]:
propiedades = pd.read_csv('../../datos_con_fecha.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', \
            'pileta', 'cochera','transporte','comercio', 'servicios', 'doble piso', 'jardin']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [4]:
# Parametros

criterios = ['mse','friedman_mse']
max_depth = [None,50,75,100,150]
features = [0.4,0.6,0.8,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 20
arbol = DecisionTreeRegressor()

In [5]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 17:33:11 --- 17:50:26 
 Precision: 96.39 
 Parametros = {'max_features': 0.8, 'criterion': 'mse', 'max_depth': 100}


In [7]:
analizar = pd.read_csv("../properati_dataset_modificado2.csv")

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict([x])[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_GridSearch_definitivo_sinfecha.csv', index = False)
joblib.dump(grid, 'algoritmos/decision_tree_definitivo_sinfecha.pkl')

['algoritmos/decision_tree_definitivo_sinfecha.pkl']

# Haciendo bagging sin fecha

In [8]:
propiedades = pd.read_csv('../../datos_con_fecha.csv')

sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

set_pruebas = pd.read_csv("../properati_dataset_modificado2.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [12]:
cant = 10
col = ['superficie','lat', 'lon','property_type_encoded']
comodidades = ['seguridad','gimnasio','aire','pileta','cochera','transporte','comercio','servicios','doble piso','jardin']

for i in range(cant):
    com = list(np.random.choice(comodidades,6,replace = False))
    columnas = col + com
    set_entrenamiento_datos = propiedades.loc[:,columnas]
    set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']

In [13]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_bagging_definitivo_sinfecha.csv', index = False)

# Grid Search y Cross Validation con columnas de descripcion

In [3]:
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

In [50]:
propiedades = pd.read_csv('../../set_datos_sup_desc.csv')

propiedades.loc[:, 'Year'] = propiedades.loc[:,'created_on'].apply(lambda x: int(x.split('-')[0]))

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', \
'pileta', 'cochera','transporte','cocina','comedor', 'living', 'banos', 'balcon','ambientes',\
'departamento', 'dormitorios','quincho', 'granito','porcelanato','completo', 'excelente', 'edificio',\
'lavadero', 'amplio', 'planta', 'cuadras', 'placard', 'terraza', 'acceso', 'parrilla', 'luminoso',\
'madera', 'credito', 'calidad', 'cucicba', 'amenities']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [51]:
# Parametros

criterios = ['mse','friedman_mse']
max_depth = [None,50,75,100,150]
features = [0.4,0.6,0.8,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 20
arbol = DecisionTreeRegressor()

In [52]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 19:24:17 --- 19:46:33 
 Precision: 93.87 
 Parametros = {'max_features': 0.8, 'criterion': 'mse', 'max_depth': 75}


In [53]:
analizar = pd.read_csv("../properati_dataset_modificado_descripciones.csv")

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict([x])[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_GridSearch_ColumnasDescripcion.csv', index = False)
joblib.dump(grid, 'algoritmos/decision_tree_ColumnasDescripcion.pkl')

['algoritmos/decision_tree_ColumnasDescripcion.pkl']

# Haciendo más columnas de descripción

In [2]:
propiedades = pd.read_csv('../../set_datos_sup_desc.csv')

sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades.loc[:, 'Year'] = propiedades.loc[:,'created_on'].apply(lambda x: int(x.split('-')[0]))

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

set_pruebas = pd.read_csv("../properati_dataset_modificado_descripciones.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [3]:
cant = 10

col = ['superficie','lat', 'lon','property_type_encoded']
comodidades = ['seguridad','gimnasio', 'aire', 'pileta', 'cochera','transporte','cocina','comedor',\
'living', 'banos', 'balcon','ambientes','departamento', 'dormitorios','quincho', 'granito','porcelanato',\
'completo', 'excelente', 'edificio','lavadero', 'amplio', 'planta', 'cuadras', 'placard', 'terraza', 'acceso',\
'parrilla', 'luminoso','madera', 'credito', 'calidad', 'cucicba', 'amenities','expensas','mesada',\
'responsable','muebles', 'espacio', 'diario', 'independiente', 'cubierta','contrafrente', 'integrada',\
'recepcion','categoria', 'profesional', 'baulera', 'natural']
comodidades = comodidades + ['unidades', 'principal', 'espacio', 'estacion', 'vestidor', 'solarium', 'central', \
                             'parquet', 'estrenar', 'granito']
comodidades = comodidades + ['patio',  'casa', 'suite', 'vista', 'hall', 'agua',  \
        'lote', 'fondo', 'acero', 'torre',  'horno',  'sala', 'salon', 'frente','toilette']
#comodidades = comodidades + ['piso', 'pisos', 'apto','sobre','doble','marca','nuevo', 'bacha', 'ideal','excelente']
comodidades = comodidades + ['afip', 'auto',  'autos', 'super', 'mejor', 'ninos', 'sauna', 'lindo', 'caba',\
                             'rio', 'gym', 'luz', 'sol', 'spa']
n = int(len(comodidades) * 0.8)

for i in range(cant):
    com = list(np.random.choice(comodidades,n,replace = False))
    columnas = col + com
    set_entrenamiento_datos = propiedades.loc[:,columnas]
    set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']
    #print(i,end = '-')

In [5]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/DecisionTree_88ColumnasDescripcion.csv', index = False)